In [7]:
import random 
from random import randint

### Exponensiación rapida

http://www-users.math.umn.edu/~garrett/crypto/Code/FastPow_Python.html 

In [1]:
def exp_mod(a,b,n):
    X = a
    E = b
    Y = 1
    while E > 0:
        if E % 2 == 0:#Es par?
            X = (X * X) % n
            E = E/2
        else:
            Y = (X * Y) % n
            E = E - 1
    return Y

In [2]:
def exp(a,b):
    if b == 0:
        return 1
    else:   
        result = a
        for i in range(b-1):
            result *=a
        return result
        

### Miller Rabin

#### Referencias:

https://gist.github.com/Ayrx/5884790 

https://www.geeksforgeeks.org/primality-test-set-3-miller-rabin/ 

https://github.com/womogenes/Miller-Rabin/blob/master/miller_rabin.py

La función power se implemento para mejorar la velocidad de computo del numero primo. Sin embargo, esta usa shift y AND, en caso de no poner utilizarlos esta funcion puede ser remplazada por exp_mod

In [47]:
def power_(x, y, p):
    res = 1;
    x = x % p;
    while (y > 0): # mayor a 0
        if (y%2)>0: #Verifico si es par
            res = (res * x) % p;#pondero por el numero y calculo el modulo
        y = y//2 #Div entera por 2
        x = (x * x) % p; #pondero y saco modulo
     
    return res;

In [48]:

def miller_rabin(n, k):
    if n == 2: #Es 2?
        return True

    if n % 2 == 0: #Es par?
        return False

    r, s = 0, n - 1
    while s % 2 == 0:
        r += 1
        s //= 2
    for i in range(k):
        a = random.randrange(2, n - 1)
        x = power_(a, s, n)
        if x == 1 or x == n - 1:
            continue
        for j in range(r - 1):
            x = power_(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True


In [43]:
def random_with_N_digits(n):
    range_start = 10**(n-1) #
    range_end = (10**n)-1 #exp(10,n) -1 #
    return randint(range_start, range_end)


In [49]:
#random_with_N_digits(5)

In [45]:
def generar_primo(l):
    while True:       
        a = random_with_N_digits(l) 
        if miller_rabin(a,100):
            return a

In [50]:
#generar_primo(100)

### Algoritmo extendido de euclides

https://www.geeksforgeeks.org/python-program-for-basic-and-extended-euclidean-algorithms-2/
    
https://www.techiedelight.com/extended-euclidean-algorithm-implementation/    

In [15]:
def gcdExtended(a, b): 
    # Caso base 
    if a == 0 :  
        return b,0,1
             
    gcd,x1,y1 = gcdExtended(b%a, a) 
     
    # Actualizo x y uso el resultado recursivo
    x = y1 - (b//a) * x1 
    y = x1 
     
    return gcd,x,y
      
def alg_ext_euclides(a,b):
    gcd = gcdExtended(a,b)
    return gcd[0]
 

### Inverso

https://www.geeksforgeeks.org/multiplicative-inverse-under-modulo-m/

In [16]:
def inverso(a, n):
     
    for x in range(1, n):
        if (((a%n) * (x%n)) % n == 1):
            return x
    return -1

### Claves Publicas y Privadas

Revisar lo de los largos y randoms

In [36]:
def generar_claves(l):
    l_ = random.randint(2, l)
    P = generar_primo(l) # l o numero random <= a l ?
    Q = generar_primo(l)
    N = P*Q
    phi_N = (P-1)*(Q-1)
    
    primo_rel = 0 #Para iniciar la variable
    busca_primo_rel = True
    while busca_primo_rel:
        primo_rel = random_with_N_digits(l) #El primo relativo 
        if alg_ext_euclides(primo_rel,phi_N) == 1:
            busca_primo_rel= False
    inverso_primo = inverso(primo_rel, phi_N)
    S = (inverso_primo, N)
    P = (primo_rel, N)
    
    #Escritura de archivo clave secreta
    fs = open('private_key', 'w')
    fs.write(str(S[0])+ "\n")
    fs.write(str(S[1])+ "\n")
    fs.close()
    
     #Escritura de archivo clave publica
    fp = open('public_key', 'w')
    fp.write(str(P[0])+ "\n")
    fp.write(str(P[1])+ "\n")
    
    return  [S, P]
            

In [40]:
#generar_claves(5)

[(1126844303, 3579095129), (14807, 3579095129)]

### Cifrado y Decifrado

In [ ]:
def enc(m):
    p = open("public_key", "r").read().splitlines()
    return pow(m, int(p[0]), int(p[1]))

In [ ]:
def dec(m):
    s = open("private_key", "r").read().splitlines()
    return pow(m, int(s[0]), int(s[1]))